## Urgent Data 
> take 2.0

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from doubledot import *
import pandas as pd
from time import sleep
import os

Directory 'atms_download' already exists.
Directory 'atms_upload' already exists.


In [3]:
sf = Salesforce()

In [4]:
### Clean out old data ?
# for table in Salesforce.model_d.keys():
#     sf.delete_sf_objects(table)

In [16]:
#| eval: false


# 415645 73012 105765 and 342765 are also good contacts to bring in when you are rea and organization 133156
pelton_ids = [ 4708, 119430, 119431, 144164,144165, 144166, 144167, 415645, 73012, 105765, 342765 ]
pelton_ids = [ 105, 867869, 774081 ]
sf.retrieve_atms_records_by_contactId(pelton_ids)

# number of objects in dictionary
assert len(sf.atms.obj_d) == len(['sales', 'contacts', 'memberships','items']), f"wrong number of objects in dictionary {sf.atms.obj_d.keys()}"
# number of files in directory
assert len(os.listdir(sf.atms.download_dir)) == len(['sales', 'contacts', 'memberships','items']), f"wrong number of files in directory {sf.atms.download_dir}"



fetch_data_by_contactIds : 105
we have contact_id 105 and obj is sales
http://crm-api-neaq.atmsplus.com/api/sales/contact/105
[{'saleKey': '5225', 'saleAmount': '0.0000', 'paymentAmount': '0.0000', 'saleDate': '2009-10-10T09:31:30.107', 'active': True, 'terminalKey': 27, 'ticketCount': 0, 'eventDate': None, 'booking': {'bookingId': 1307, 'bookingContactKey': 3481, 'bookingContactType': 'Primary', 'contactKey': 105, 'contactIndividualKey': 104, 'contactOrganizationKey': 0, 'displayName': 'Lee, Sherry', 'firstName': 'Sherry', 'lastName': 'Lee', 'email': 'twofishmom@aol.com', 'phone': '6174690341'}, 'saleComment': None, 'saleDetails': [{'saleDetailId': 13499, 'saleId': 5225, 'itemKey': 2, 'scheduleKey': 7916, 'rateKey': 6, 'categoryKey': 67, 'itemQuantity': 1, 'pricingPriceKey': 1214, 'itemPrice': 8.95, 'itemTotal': 8.95, 'couponTotal': 0.0, 'discountTotal': 0.0, 'total': 8.95, 'revenueDate': '2009-10-10T10:00:00', 'refundReasonKey': None, 'systemPriceOverride': 'N', 'membershipTermKey': 

In [97]:
#| eval: false

# perfect_data is only necessary to remove extrastuff
# perfect_data does two things, and it shouldn't: it returns a dict AND it writea a file. Pick a lane!
df_d = sf.perfect_data()
# for k, v in _d.items():
#     display(v)

Directory 'atms_download' already exists.
Directory 'atms_upload' already exists.


TypeError: Salesforce.perfect_data() missing 1 required positional argument: 'df_d'

In [102]:
import jmespath as jp

OG_memMembers_s = "[].membershipTerms[].membershipMembers[].{\
membershipMemberId__c:membershipMemberId,\
membershipTermKey__r_1_membershipTermId__c:membershipTermKey,\
cardNumber__c:cardNumber,\
membershipNumber__c:membershipNumber,\
cardStatus__c:cardStatus,\
contactKey__r_1_contactId__c:contactKey,\
displayName__c:displayName}"

s = "membershipMemberId__c:membershipMemberId,\
membershipTermKey__r_1_membershipTermId__c:membershipTermKey,\
cardNumber__c:cardNumber,\
membershipNumber__c:membershipNumber,\
cardStatus__c:cardStatus,\
contactKey__r_1_contactId__c:contactKey,\
displayName__c:displayName"

# os = "[].{id: membershipId, MTs:membershipTerms[].{id: membershipTermId, MMs:membershipMembers[].{id: membershipMemberId," + s+ "}}}}"
print(os)
memMembers_s = "[].{Mid: membershipId, MTs:membershipTerms[].{MTid: membershipTermId }}"
memMembers_s = "[].{Mid: membershipId, MTs:membershipTerms[]}"
memMembers_s = "[].\
{id: membershipId, MTs:membershipTerms[].\
    {id: membershipTermId, MMs:membershipMembers[].\
        {membershipMemberId__c:membershipMemberId,\
        membershipTermKey__r_1_membershipTermId__c:membershipTermKey,\
        cardNumber__c:cardNumber,\
        membershipNumber__c:membershipNumber,\
        cardStatus__c:cardStatus,\
        contactKey__r_1_contactId__c:contactKey,\
        displayName__c:displayName\
}}}"

_l = jp.search(memMembers_s , sf.atms.obj_d['memberships'])
_l

[].{id: membershipId, MTs:membershipTerms[].{id: membershipTermId, MMs:membershipMembers[].{id: membershipMemberId,membershipMemberId__c:membershipMemberId,membershipTermKey__r_1_membershipTermId__c:membershipTermKey,cardNumber__c:cardNumber,membershipNumber__c:membershipNumber,cardStatus__c:cardStatus,contactKey__r_1_contactId__c:contactKey,displayName__c:displayName}}}}


[{'id': 13,
  'MTs': [{'id': 13,
    'MMs': [{'membershipMemberId__c': 13,
      'membershipTermKey__r_1_membershipTermId__c': 13,
      'cardNumber__c': 1,
      'membershipNumber__c': '00001301',
      'cardStatus__c': 'Active',
      'contactKey__r_1_contactId__c': 105,
      'displayName__c': 'Lee, Sherry'}]},
   {'id': 118327,
    'MMs': [{'membershipMemberId__c': 13,
      'membershipTermKey__r_1_membershipTermId__c': 118327,
      'cardNumber__c': 1,
      'membershipNumber__c': '00001301',
      'cardStatus__c': 'Active',
      'contactKey__r_1_contactId__c': 105,
      'displayName__c': 'Lee, Sherry'}]},
   {'id': 194108,
    'MMs': [{'membershipMemberId__c': 13,
      'membershipTermKey__r_1_membershipTermId__c': 194108,
      'cardNumber__c': 1,
      'membershipNumber__c': '00001301',
      'cardStatus__c': 'Active',
      'contactKey__r_1_contactId__c': 105,
      'displayName__c': 'Lee, Sherry'}]},
   {'id': 243431,
    'MMs': [{'membershipMemberId__c': 13,
      'members

In [104]:
new_l = []
for m_d in _l:
    for t_d in m_d['MTs']:
        new_mt_id = ('_'.join([str(i) for i in [m_d['id'],t_d['id']]]))
        for mm_d in t_d['MMs']:
            new_mm_id = ('_'.join([str(i) for i in [m_d['id'],t_d['id'],mm_d['membershipMemberId__c']]]))
            # print(new_mm_id)
            new_d = mm_d.copy()
            new_d['membershipMemberId__c'] = new_mm_id
            new_d['membershipTermKey__r_1_membershipTermId__c'] = new_mt_id
            new_l.append(new_d)
new_l    

[{'membershipMemberId__c': '13_13_13',
  'membershipTermKey__r_1_membershipTermId__c': '13_13',
  'cardNumber__c': 1,
  'membershipNumber__c': '00001301',
  'cardStatus__c': 'Active',
  'contactKey__r_1_contactId__c': 105,
  'displayName__c': 'Lee, Sherry'},
 {'membershipMemberId__c': '13_118327_13',
  'membershipTermKey__r_1_membershipTermId__c': '13_118327',
  'cardNumber__c': 1,
  'membershipNumber__c': '00001301',
  'cardStatus__c': 'Active',
  'contactKey__r_1_contactId__c': 105,
  'displayName__c': 'Lee, Sherry'},
 {'membershipMemberId__c': '13_194108_13',
  'membershipTermKey__r_1_membershipTermId__c': '13_194108',
  'cardNumber__c': 1,
  'membershipNumber__c': '00001301',
  'cardStatus__c': 'Active',
  'contactKey__r_1_contactId__c': 105,
  'displayName__c': 'Lee, Sherry'},
 {'membershipMemberId__c': '13_243431_13',
  'membershipTermKey__r_1_membershipTermId__c': '13_243431',
  'cardNumber__c': 1,
  'membershipNumber__c': '00001301',
  'cardStatus__c': 'Active',
  'contactKey__

In [107]:
memMembers_keys =  [ "membershipMemberId__c", 
"membershipTermKey__r.membershipTermId__c", 
"cardNumber__c", 
"membershipNumber__c", 
"cardStatus__c", 
"contactKey__r.contactId__c", 
"displayName__c" ]
from doubledot.crema_sf import write_jmespath_to_csv 
write_jmespath_to_csv(new_l, 'debug.csv', memMembers_keys)

Directory 'atms_download' already exists.
Directory 'atms_upload' already exists.
Writing 35 records to debug.csv
13_13_13	13_13	1	00001301	Active	105	Lee, Sherry

13_118327_13	13_118327	1	00001301	Active	105	Lee, Sherry

13_194108_13	13_194108	1	00001301	Active	105	Lee, Sherry

13_243431_13	13_243431	1	00001301	Active	105	Lee, Sherry

13_243431_243288	13_243431	2	00001302	Active	380046	McCain, Andrea

13_256286_243288	13_256286	2	00001302	Active	380046	McCain, Andrea

13_256286_13	13_256286	1	00001301	Active	105	Lee, Sherry

13_294095_13	13_294095	1	00001301	Active	105	Lee, Sherry

13_294095_243288	13_294095	2	00001302	Active	380046	McCain, Andrea

13_316898_13	13_316898	1	00001301	Active	105	Lee, Sherry

13_316898_243288	13_316898	2	00001302	Active	380046	McCain, Andrea

13_355807_13	13_355807	1	00001301	Active	105	Lee, Sherry

13_355807_243288	13_355807	2	00001302	Active	380046	McCain, Andrea

13_381793_13	13_381793	1	00001301	Active	105	Lee, Sherry

13_381793_243288	13_381793	2	000

In [49]:
#| eval: false
# print(df_d.keys())
df_d = {}
for i in Salesforce.model_d.keys(): 
    full_path = os.path.join(Salesforce.class_upload_dir, i+'.csv')
    if os.path.isfile(full_path):
        df_d[i] = pd.read_csv(full_path, sep='\t')

df2_d = sf.perfect_data(df_d)


Salesforce.process_objects: sf_object_s : Contact
Writing 3 records to /Users/josephmann/Documents/Github/doubledot/sf_upload/Contact.csv
Salesforce.process_objects: sf_object_s : Membership__c
Writing 3 records to /Users/josephmann/Documents/Github/doubledot/sf_upload/Membership__c.csv
Writing 19 records to /Users/josephmann/Documents/Github/doubledot/sf_upload/MembershipTerm__c.csv
Writing 35 records to /Users/josephmann/Documents/Github/doubledot/sf_upload/MembershipMember__c.csv
Salesforce.process_objects: sf_object_s : MembershipTerm__c
Writing 3 records to /Users/josephmann/Documents/Github/doubledot/sf_upload/Membership__c.csv
Writing 19 records to /Users/josephmann/Documents/Github/doubledot/sf_upload/MembershipTerm__c.csv
Writing 35 records to /Users/josephmann/Documents/Github/doubledot/sf_upload/MembershipMember__c.csv
Salesforce.process_objects: sf_object_s : MembershipMember__c
Writing 3 records to /Users/josephmann/Documents/Github/doubledot/sf_upload/Membership__c.csv
Wr

Exception: bad lookup

In [35]:
df_d

{}

In [27]:
sf.write_dict_to_csv(df_d ) # only necessary if generate_upload_files=Fals

 writing Contact to /Users/josephmann/Documents/Github/doubledot/sf_upload/Contact.csv
 writing Membership__c to /Users/josephmann/Documents/Github/doubledot/sf_upload/Membership__c.csv
 writing MembershipTerm__c to /Users/josephmann/Documents/Github/doubledot/sf_upload/MembershipTerm__c.csv
 writing MembershipMember__c to /Users/josephmann/Documents/Github/doubledot/sf_upload/MembershipMember__c.csv
 writing Sale__c to /Users/josephmann/Documents/Github/doubledot/sf_upload/Sale__c.csv
 writing SaleDetail__c to /Users/josephmann/Documents/Github/doubledot/sf_upload/SaleDetail__c.csv
 writing Ticket__c to /Users/josephmann/Documents/Github/doubledot/sf_upload/Ticket__c.csv


In [110]:
sf.clean_upload_dir()


Directory 'atms_download' already exists.
Directory 'atms_upload' already exists.


In [120]:

sf.upload_csv_to_sf(clean_all=False, generate_upload_files=True)

Contact contactId__c
execute_job
Salesforce.process_objects: sf_object_s : Contact
Writing 3 records to /Users/josephmann/Documents/Github/doubledot/sf_upload/Contact.csv
Lee	Sherry	02467-3661	Chestnut Hill	219 Grove St	United States			105

Rodrigues	Sai	02120	Boston	102 Hammond St Apt 2	United States			774081

Holman	Allison	02114-2137	Boston	234 Causeway Street  911	United States			867869

Created job 7508Y00000nIslnQAC for Contact with operation upsert
Uploading job 7508Y00000nIslnQAC of object Contact
Uploaded job 7508Y00000nIslnQAC of object Contact with status 201
response.text : 
 : 
close_job response.text :
 {"id":"7508Y00000nIslnQAC","operation":"upsert","object":"Contact","createdById":"0058Y00000CIEn9QAH","createdDate":"2023-05-30T04:25:52.000+0000","systemModstamp":"2023-05-30T04:25:52.000+0000","state":"UploadComplete","externalIdFieldName":"contactId__c","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":57.0}
pre loop job status: {'id': '7508Y00000nIslnQAC', 

In [19]:
sf.execute_job('MembershipMember__c', 'upsert', external_id=sf.model_d['MembershipMember__c']['external_id'], use_ATMS_data=False) 

execute_job
Salesforce.process_objects: sf_object_s : MembershipMember__c
Created job 7508Y00000nIpQ2QAK for MembershipMember__c with operation upsert
Uploading job 7508Y00000nIpQ2QAK of object MembershipMember__c
Uploaded job 7508Y00000nIpQ2QAK of object MembershipMember__c with status 201
response.text : 
 : 
close_job response.text :
 {"id":"7508Y00000nIpQ2QAK","operation":"upsert","object":"MembershipMember__c","createdById":"0058Y00000CIEn9QAH","createdDate":"2023-05-29T19:46:22.000+0000","systemModstamp":"2023-05-29T19:46:22.000+0000","state":"UploadComplete","externalIdFieldName":"membershipMemberId__c","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":57.0}
pre loop job status: {'id': '7508Y00000nIpQ2QAK', 'operation': 'upsert', 'object': 'MembershipMember__c', 'createdById': '0058Y00000CIEn9QAH', 'createdDate': '2023-05-29T19:46:22.000+0000', 'systemModstamp': '2023-05-29T19:46:26.000+0000', 'state': 'JobComplete', 'externalIdFieldName': 'membershipMemberId__c', 'c

## Problems ...


In [ ]:
#|eval: false

# why does saleId__c 889832 not have any tickets or saleDetail pointing at it? or me: 82192 323674 (I am 73012)
# why does this point from saleId and not saleKey?
from doubledot.crema_sf import fields_pointing_to_foreign_key, get_pointing_foreign_key_values,reduce_to_referenced_rows

_df = df_d['Sale__c']
print(_df.loc[_df['saleId__c'] == 323674, :])

print(get_pointing_foreign_key_values('saleId__c', df_d))
pointing = get_pointing_foreign_key_values('saleId__c', df_d)
print(len(pointing))

print(_df['saleId__c'].unique())
externals = _df['saleId__c'].unique()

pointing.intersection(externals)
      

In [ ]:
reduce_to_referenced_rows(df_d)


In [ ]:
_d['Ticket__c']['saleKey__r.saleId__c']